## Fit material properties Vs mc to obtain functions for UMAT

### Imports and working path
Run the following section to import the required libraries and set the working directory. 

In [ ]:
# Imports
from math import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Math
from sklearn.metrics import r2_score
import os
import mpmath as mp
mp.dps = 150 # increase precision to stabilize matrix operations (default = 15)
np.set_printoptions(suppress=True)

# Set working path
runpath = os.path.dirname(os.getcwd()) + '/'  # main path
print(f"Working on {runpath}")

### Extract engineering constants for varying moisture content

In [ ]:
def read_tissue_engineering_constants(mainpath, tissue):
    # Path to compliance file   
    csv_path = os.path.join(mainpath, tissue, f"{tissue}_elastic_compliance_coeffs.csv") 
    # Read file
    df = pd.read_csv(csv_path, sep=None, engine='python')  # auto-detect delimiter
    # Pull out compliance coefficients
    S11 = df["D11"]
    S22 = df["D22"]
    S33 = df["D33"]
    S44 = df["D44"]
    S55 = df["D55"]
    S66 = df["D66"]
    S12 = df["D12"]
    S13 = df["D13"]
    S23 = df["D23"] 
    # Compute engineering constants
    E1 = 1.0 / S11
    E2 = 1.0 / S22
    E3 = 1.0 / S33
    
    G23 = 1.0 / S44
    G13 = 1.0 / S55
    G12 = 1.0 / S66
    
    nu12 = -S12 / S11
    nu13 = -S13 / S11
    nu23 = -S23 / S22
    
    # Assemble into a DataFrame
    data = {
        "w": 0.12,
        "tissue": tissue,
        "E1":     E1,
        "E2":     E2,
        "E3":     E3,
        "v12":    nu12,
        "v13":    nu13,
        "v23":    nu23,
        "G12":    G12,
        "G13":    G13,
        "G23":    G23,
    }
    return pd.DataFrame(data)

# Calculate engineering constants for each tissue
tissues = ["EW", "TW", "LW"]
df_list = [read_tissue_engineering_constants(runpath, t) for t in tissues]
df_all = pd.concat(df_list, ignore_index=True)
display(df_all)

### Fit material properties to moisture content

In [ ]:
# Set Seaborn style
sns.set_style("whitegrid")
# Set fontsizes
titlesize = 18
labelsize = 14

def fit_polynomial(x, y, degree):
    # Fit polynomial
    coeffs = np.polyfit(x, y, degree)
    poly_eq = np.poly1d(coeffs)
    fitted_y = poly_eq(x)
    r2 = r2_score(y, fitted_y)

    return coeffs, fitted_y, r2
# end: fit_polynomial

def plot_fit_mat_properties(dataset, titlesize, labelsize, poly_degree_dict):
    layers = dataset['layer'].unique()
    markersize = 2
    for layer in layers:
        df_layer = dataset[dataset['layer'] == layer]
        poly_degree = poly_degree_dict[layer]
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

        # --- Subplot 1: Engineering Constants ---
        ax1.set_xlabel("mc [-]", fontsize=labelsize)
        ax1.tick_params(labelsize=labelsize)
        ax1.tick_params(axis='y', labelcolor='blue')
        ax1.set_ylabel(r"$E_{ii}$ [GPa]", color='blue', fontsize=labelsize)

        # Plot Young's moduli (primary y-axis)
        for var, color, label in zip(["E1", "E2", "E3"],
                                     ["blue", "cyan", "navy"],
                                     [r"${E_1}$", r"${E_2}$", r"${E_3}$"]):
            # Plot data
            ax1.plot(df_layer["mc"], df_layer[var], 'o', label=label, color=color, markersize=markersize)
            # Plot polynomial fit
            coeffs, y_fit, r2 = fit_polynomial(df_layer["mc"], df_layer[var], poly_degree)
            ax1.plot(df_layer["mc"], y_fit, '-', color=color)
            #print(var, np.polyval(coeffs, 0.12))
        
        # Plot Poisson's ratios and shear moduli (secondary y-axis)
        ax3 = ax1.twinx()
        ax3.set_ylabel(r"$\nu_{ij}$ [-], $10^{-1} \times G_{ij}$ [GPa]", color='red', fontsize=labelsize)

        for var, color, label in zip(["v23", "v13", "v12", "G23", "G13", "G12"],
                                     ["red", "pink", "orange", "purple", "magenta", "brown"],
                                     [r"${\nu_{23}}$", r"${\nu_{13}}$", r"${\nu_{12}}$", r"${G_{23}}$", r"${G_{13}}$", r"${G_{12}}$"]):
            y_data = df_layer[var] / 10 if "G" in var else df_layer[var]
            ax3.plot(df_layer["mc"], y_data, 'o', label=label, color=color, markersize=markersize)
            # Plot polynomial fit
            coeffs, y_fit, r2 = fit_polynomial(df_layer["mc"], df_layer[var], poly_degree)
            y_fit = y_fit / 10 if "G" in var else y_fit
            ax3.plot(df_layer["mc"], y_fit, '-', color=color)
            #print(var, np.polyval(coeffs, 0.12))
            
        ax3.tick_params(axis='y', labelcolor='red', labelsize=labelsize)

        # --- Grid Alignment ---
        ax1.set_xlim(left=min(df_layer["mc"]), right=max(df_layer["mc"]))
        ax1.yaxis.set_major_locator(plt.MaxNLocator(nbins=5))
        ax3.yaxis.set_major_locator(plt.MaxNLocator(nbins=5))
        ax1.set_ylim(bottom=max(0.01, ax1.get_ylim()[0]))
        ax3.set_ylim(bottom=max(0.01, ax3.get_ylim()[0]))
        ax1.set_yticks(np.linspace(ax1.get_ylim()[0], ax1.get_ylim()[1], num=6))
        ax3.set_yticks(np.linspace(ax3.get_ylim()[0], ax3.get_ylim()[1], num=6))
        ax1.grid(True, linestyle='-', color='grey', linewidth=0.5, alpha=0.7)
        ax3.grid(False)
        
        # --- Subplot 2: Hygroexpansion Coefficients ---
        ax2.set_xlabel("mc [-]", fontsize=labelsize)
        ax2.tick_params(labelsize=labelsize)
        ax2.set_ylabel(r"$\alpha_{1}$ [-]", color='green', fontsize=labelsize)

        # Plot alpha1 (primary y-axis)
        ax2.plot(df_layer["mc"], df_layer["alpha1"], 'o', label=r"${\alpha_1}$", color="green", markersize=markersize)
        # Plot polynomial fit
        coeffs, y_fit, r2 = fit_polynomial(df_layer["mc"], df_layer["alpha1"], poly_degree)
        ax2.plot(df_layer["mc"], y_fit, '-', color=color)
        ax2.plot(df_layer["mc"], y_fit, '-', color="green")
        ax2.tick_params(axis='y', labelcolor='green')

        # Plot alpha2 and alpha3 (secondary y-axis)
        ax4 = ax2.twinx()
        for var, color, label in zip(["alpha2", "alpha3"], 
                                     ["olive", "darkgreen"], 
                                     [r"${\alpha_2}$", r"${\alpha_3}$"]):
            ax4.plot(df_layer["mc"], df_layer[var], 'o', label=label, color=color, markersize=markersize)
            # Plot polynomial fit
            coeffs, y_fit, r2 = fit_polynomial(df_layer["mc"], df_layer[var], poly_degree)
            ax4.plot(df_layer["mc"], y_fit, '-', color=color)
            
        ax4.tick_params(axis='y', labelcolor='olive', labelsize=labelsize)

        # --- Grid Alignment ---
        ax2.set_xlim(left=min(df_layer["mc"]), right=max(df_layer["mc"]))
        ax2.set_ylim(0, 0.2)
        ax4.set_ylim(0.2, 0.5)
        ax2.yaxis.set_major_locator(plt.MaxNLocator(nbins=5))
        ax4.yaxis.set_major_locator(plt.MaxNLocator(nbins=5))
        ax2.set_yticks(np.linspace(ax2.get_ylim()[0], ax2.get_ylim()[1], num=6))
        ax4.set_yticks(np.linspace(ax4.get_ylim()[0], ax4.get_ylim()[1], num=6))
        ax2.grid(True, linestyle='-', color='grey', linewidth=0.5, alpha=0.7)
        ax4.grid(False)

        # --- Legends ---
        handles_ec, labels_ec = ax1.get_legend_handles_labels()
        handles_vg, labels_vg = ax3.get_legend_handles_labels()
        ax1.legend(handles=handles_ec + handles_vg, labels=labels_ec + labels_vg, loc="upper right", fontsize=labelsize)

        handles_alpha, labels_alpha = ax2.get_legend_handles_labels()
        handles_alpha2_3, labels_alpha2_3 = ax4.get_legend_handles_labels()
        ax2.legend(handles=handles_alpha + handles_alpha2_3, labels=labels_alpha + labels_alpha2_3, loc="upper right", fontsize=labelsize)

        fig.suptitle(f"{layer}", fontsize=titlesize, fontweight='bold')
        plt.tight_layout()
    
    return [ax1, ax3, ax2, ax4]
# end: plot_fit_mat_properties

# Plot material properties and fitting
poly_degree = 10
poly_degree_dict = {
    "EW": poly_degree,
    "TW": poly_degree,
    "LW": poly_degree
}

# Check number of unique moisture values
unique_mc = df_all['w'].unique()
if len(unique_mc) < 2:
    print("Only one moisture content value (w={}) in the dataset. No fitting plot will be generated.".format(unique_mc[0]))
else:
    plot_fit_mat_properties(df_all, titlesize, labelsize, poly_degree_dict)
    plt.show()

### Save material properties in fortran format

In [ ]:
# Function to write Fortran file for engineering constants
def write_eng_const_file(filepath, dataset, poly_degree_dict):
    properties = [('E1', 'E1'), ('E2', 'E2'), ('E3', 'E3'),
                  ('v23', 'nu23'), ('v13', 'nu13'), ('v12', 'nu12'),
                  ('G23', 'G23'), ('G13', 'G13'), ('G12', 'G12')]
    
    layers = dataset['tissue'].unique()

    for idx, layer in enumerate(layers):
        filename = filepath + "/" + layer + "_eng_const.inc"
        #print(filename)
        with open(filename, 'w') as f:
            f.write("! Engineering constants\n\n")
        
            """if idx == 0:
                f.write("\tIF (MATNAME == '{}') THEN\n".format(layer))
            else:
                f.write("\tELSE IF (MATNAME == '{}') THEN\n".format(layer))"""

            poly_degree = poly_degree_dict[layer]
            df_layer = dataset[dataset['tissue'] == layer]

            # Loop over each property
            for var, label in properties:
                unique_w = df_layer['w'].unique()
                if len(unique_w) > 1:
                    # More than one moisture value: perform polynomial fitting.
                    coeffs, _, _ = fit_polynomial(df_layer['w'], df_layer[var], poly_degree)
                    poly_terms = []
                    for i, c in enumerate(coeffs):
                        deg = poly_degree - i
                        if deg == 0:
                            term = "({:.3f}D0)".format(c)
                        elif deg == 1:
                            term = "({:.3f}D0)*(KW_MC)".format(c)
                        else:
                            term = "({:.3f}D0)*(KW_MC)**{}".format(c, deg)
                        poly_terms.append(term)

                    line = "\t  KW_" + label + " = "
                    cut1 = 3
                    cut2 = 6
                    for i, term in enumerate(poly_terms):
                        if i == cut1 or i == cut2:
                            line += "+\n"
                            f.write(line)
                            # For continuation lines, use proper spacing:
                            line = "     1  " if i == cut1 else "     2  "
                        if i > 0 and i not in [cut1, cut2]:
                            line += "+ " + term + " "
                        else:
                            line += term + " "
                    f.write(line + "\n")
                else:
                    # Only one moisture value: simply use the constant value.
                    const_value = df_layer[var].iloc[0]
                    line = "\t  KW_" + label + " = ({:.3f}D0)".format(const_value)
                    f.write(line + "\n")
            f.write("\n")
    return

# end: write_eng_const_file

# Generate files for engineering constants
savepath = runpath + "Modules" 
write_eng_const_file(savepath, df_all, poly_degree_dict)
print(f"Files generated in: {savepath}")